# HistGradientBoostingRegressor を ONNX に変換した場合の精度評価 (時系列)

目的: scikit-learn の HistGradientBoostingRegressor を ONNX に変換し、精度を比較する
範囲: ONNX 変換による精度の変化を定量的に評価する

## データセット
- データ: statsmodels から "Atmospheric CO2" (Mauna Loa Observatory) を使用
- 変数: 月次 CO2 濃度 (ppm) の時系列データを使用する
- 備考: Kaggle のデータセットを使用する場合も同様の手順でダウンロード・前処理を行う

### Kaggle データセットを使用する場合の手順
1. Kaggle にログインして Profile > Account > Create New API Token で `kaggle.json` をダウンロード
2. `~/.kaggle/kaggle.json` (Windows は `%USERPROFILE%/.kaggle/kaggle.json`) に配置し、パーミッションを 600 に設定
3. `pip install kaggle` で `kaggle competitions download -c <name>` や `kaggle datasets download <user>/<dataset>` を使用する
4. ダウンロードしたデータを解凍し、CSV ファイルを読み込んで前処理を行う

## 手順 (処理フロー)
1. ライブラリのインポート
2. CO2 データの読み込みと前処理
3. 時系列データを教師あり学習用に変換
4. HistGradientBoostingRegressor で学習
5. ONNX 変換し ONNX Runtime で推論
6. scikit-learn (float64/float32) と ONNX Runtime の精度を比較

In [1]:
# 1. 必要なライブラリのインポート
# 以下のライブラリを使用して処理を進める
import numpy as np            # 数値計算・配列操作
import pandas as pd           # データフレーム操作
import statsmodels.api as sm  # CO2 データセットの読み込み
from pathlib import Path      # パス操作
from sklearn.ensemble import HistGradientBoostingRegressor  # 勾配ブースティング回帰
from sklearn.metrics import mean_absolute_error, root_mean_squared_error  # 評価指標 (MAE, RMSE)
import onnx                   # ONNX モデルの操作
import onnxruntime as ort     # ONNX Runtime 推論
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType      # ONNX のデータ型定義


In [6]:
# 2. データの読み込みと前処理
# - statsmodels から CO2 データセットを読み込む
# - 欠損値を処理し、時系列データとして整える
co2 = sm.datasets.co2.load_pandas().data

# インデックスを datetime 型にしておく（ほぼなっているはずだが念のため）
co2.index = pd.to_datetime(co2.index) # 日付を datetime 型に
co2 = co2.sort_index()
co2['co2'] = co2['co2'].ffill()                     # 欠損値を前の値で補完
co2_series = co2['co2'].to_numpy(dtype=np.float64)  # ndarray (float64) に変換
print(f'データ数: {len(co2_series)}')
co2.head()


データ数: 2284


,co2
1958-03-29,316.1
1958-04-05,317.3
1958-04-12,317.6
1958-04-19,317.5
1958-04-26,316.4


In [8]:
# 3. 時系列データを教師あり学習用に変換
# 過去の値を特徴量として使用する関数を定義
from typing import Iterable, Tuple

def series_to_supervised(series: Iterable[float], lags: int) -> Tuple[np.ndarray, np.ndarray]:
    """時系列データを lags 個の過去の値を使って教師あり学習用に変換する
    例: lags=3 のとき [x0, x1, x2] -> y3, [x1, x2, x3] -> y4 のように変換
    """
    arr = np.asarray(series, dtype=np.float64).ravel()  # 1次元の ndarray に変換
    X, y = [], []
    for i in range(lags, len(arr)):
        X.append(arr[i - lags : i])  # 過去 lags 個の値を特徴量に
        y.append(arr[i])            # 現在の値を目的変数に
    return np.vstack(X), np.asarray(y)

def split_train_test(X: np.ndarray, y: np.ndarray, test_size: float):
    """時系列データを時系列順に訓練データとテストデータに分割する"""
    n_test = max(1, int(len(X) * test_size))
    split = len(X) - n_test
    return X[:split], X[split:], y[:split], y[split:]


In [9]:
# 4. 特徴量・目的変数の生成と訓練/テスト分割
lags = 48            # 過去 1 年間（12ヶ月×4週）のデータを特徴量に
test_ratio = 0.2     # テストデータ 20% に設定
X, y = series_to_supervised(co2_series, lags=lags)
X_train, X_test, y_train, y_test = split_train_test(X, y, test_size=test_ratio)
print(f'X_train: {X_train.shape}, X_test: {X_test.shape}')


X_train: (1789, 48), X_test: (447, 48)


In [10]:
# 5. モデル学習 (HistGradientBoostingRegressor)
# - 時系列予測に適したパラメータを設定
# - random_state を固定して再現性を確保
model = HistGradientBoostingRegressor(
    max_depth=6,
    learning_rate=0.05,
    max_iter=400,
    l2_regularization=0.0,
    random_state=0,
)
model.fit(X_train, y_train)


,loss,'squared_error'
,quantile,None
,learning_rate,0.05
,max_iter,400
,max_leaf_nodes,31
,max_depth,6
,min_samples_leaf,20
,l2_regularization,0.0
,max_features,1.0
,max_bins,255
,categorical_features,'from_dtype'


In [11]:
# 6. ONNX 変換
# - initial_types で入力の型を指定 (float32, Noneはバッチサイズが可変)
# - target_opset で使用する ONNX オペレーションセットのバージョンを指定
initial_types = [("float_input", FloatTensorType([None, X_train.shape[1]]))]
onnx_model = convert_sklearn(model, initial_types=initial_types, target_opset=17)
# ONNX Runtime の推論セッションを作成
sess = ort.InferenceSession(onnx_model.SerializeToString(), providers=["CPUExecutionProvider"])
onnx_input_name = sess.get_inputs()[0].name


In [13]:
# 7. 精度評価
# - scikit-learn モデルで float64 と float32 の両方で予測を実行
# - ONNX Runtime は float32 のみで推論
def eval_preds(y_true, y_pred):
    return {
        "rmse": float(root_mean_squared_error(y_true, y_pred)),
        "mae": float(mean_absolute_error(y_true, y_pred)),
    }

# scikit-learn (float64 / float32)
preds_sklearn64 = model.predict(X_test)
preds_sklearn32 = model.predict(X_test.astype(np.float32))

# ONNX Runtime (float32 のみ)
preds_onnx = sess.run(None, {onnx_input_name: X_test.astype(np.float32)})[0].ravel()

metrics = {
    "sklearn_float64": eval_preds(y_test, preds_sklearn64),
    "sklearn_float32": eval_preds(y_test, preds_sklearn32),
    "onnx_runtime": eval_preds(y_test, preds_onnx),
    "differences": {
        "mean_abs_diff_to_sklearn64": float(np.mean(np.abs(preds_sklearn64 - preds_onnx))),
        "max_abs_diff_to_sklearn64": float(np.max(np.abs(preds_sklearn64 - preds_onnx))),
        "mean_abs_diff_to_sklearn32": float(np.mean(np.abs(preds_sklearn32 - preds_onnx))),
        "max_abs_diff_to_sklearn32": float(np.max(np.abs(preds_sklearn32 - preds_onnx))),
    },
}
metrics


{'sklearn_float64': {'rmse': 6.898229106192108, 'mae': 5.488819023460761},
 'sklearn_float32': {'rmse': 6.898189054607836, 'mae': 5.48792300880167},
 'onnx_runtime': {'rmse': 6.898225197022421, 'mae': 5.4888140992030205},
 'differences': {'mean_abs_diff_to_sklearn64': 6.985622333251915e-06,
  'max_abs_diff_to_sklearn64': 2.991178973843489e-05,
  'mean_abs_diff_to_sklearn32': 0.005225158780008132,
  'max_abs_diff_to_sklearn32': 0.20343803337823374}}

In [14]:
# 8. 結果保存 (ONNX と metrics.json)
# - ONNX モデルをファイルに保存
# - 評価結果を JSON ファイルとして保存
import json
output_dir = Path("scikit_learn_onnx_repo/onnx_conversion_accuracy_evaluation/outputs")
output_dir.mkdir(parents=True, exist_ok=True)
onnx_path = output_dir / "hist_gradient_boosting_co2.onnx"
metrics_path = output_dir / "metrics_co2.json"
with open(onnx_path, "wb") as f:
    f.write(onnx_model.SerializeToString())
with open(metrics_path, "w", encoding="utf-8") as f:
    json.dump(metrics, f, indent=2, ensure_ascii=False)
print(f"ONNX保存先: {onnx_path}")
print(f"メトリクス保存先: {metrics_path}")


ONNX保存先: scikit_learn_onnx_repo\onnx_conversion_accuracy_evaluation\outputs\hist_gradient_boosting_co2.onnx
メトリクス保存先: scikit_learn_onnx_repo\onnx_conversion_accuracy_evaluation\outputs\metrics_co2.json
